In [1]:
import sys  
sys.path.insert(1, '.\..\..\SOURCES\.')

import random

random.seed(10)

In [2]:
from DATA import ContinousXOR
from Configuartions import IPReservoirConfiguration, ReservoirConfiguration
from Metrics import *
from IPMask import IPMask
import itertools

N_UNITS_LIST = [50, 100, 150, 200, 250]
p_range = [1,2,5,10]
tau_range = [3,4,6]
repetitions = 2

In [3]:
def evaluate_with_lambda_search(model: Reservoir, data: TimeseriesDATA, metrics: list[Metric]= [MSE()], transient = 100, lambda_range = [0, 0.05, 0.1, 0.15, 0.2]): 
    data.split([80, 10, 10])
    X_TR, Y_TR = data.TR()
    X_VAL, Y_VAL = data.VAL()

    best_lambda=0
    best_err = None
    metric = metrics[0]
    for lambda_thikonov in lambda_range:
        esn = EchoStateNetwork(model)
        esn.train(X_TR, Y=Y_TR, transient=transient, lambda_thikonov=lambda_thikonov)
        error = metric.evaluate(esn.predict(X_VAL), Y_VAL)
        if  best_err == None or best_err > error :
            best_err = error
            best_lambda = lambda_thikonov

    data.split([90,0,10])
    X_TR, Y_TR = data.TR()
    X_TS, Y_TS = data.TS()

    esn = EchoStateNetwork(model)
    esn.train(X_TR, Y=Y_TR, transient=transient, lambda_thikonov=best_lambda)
    y_pred = esn.predict(X_TS)
    return list(np.array([metric.evaluate(y_pred, Y_TS) for metric in metrics])), best_lambda


In [4]:
p_range

[1, 2, 5, 10]

In [5]:
tau_range

[3, 4, 6]

In [6]:
data = ContinousXOR(p=1)
data.delay_timeseries(1)
data.split([80,0,20])   

X_TR_INIT, _ = data.TR()
X_TR, Y_TR = data.TR()


mask_names = ["Vanilla", "Gaussian", "Hard Bimodal", "Soft Trimodal", "Edgy trimodal" ]

intrinsic_metrics: list[IntrinsicMetric] = [DeltaPhi(), MC(), MLLE() ,Neff(), Rho(), KL(X_TR_INIT[0:2000])]
estrinsic_metrics: list[EstrinsicMetric] = [MSE(), NRMSE()]
models: list[Reservoir] = []
lambda_range = [0, 0.05, 0.1, 0.15, 0.2]

#intrinsic_column_names =  ["Name", "N", "Model Index"] + [metric.name for metric in intrinsic_metrics]
#intrinsic_results = []

estrinsic_column_names = ["Name", "N", "Model Index", "p", "Tau", "Lambda"] +  [metric.name for metric in estrinsic_metrics]
estrinsic_results=[]

configs = []

for r in range(repetitions):
    print("Repetiotion:", r)
    for N in N_UNITS_LIST:
        init_point = ReservoirConfiguration(N=N ,desired_rho=0.75, input_scaling=0.5, bias=True, bh_scaling=0.1, bu_scaling=0.1,  name=f"Vanilla") 
        configs.append(init_point)
        model = init_point.build_up_model()
        #partial_intrinsic_results = [mask_names[0], N, 0] + list(np.array([metric.evaluate(model) for metric in intrinsic_metrics]))
        #intrinsic_results.append(partial_intrinsic_results)

        for p,tau in itertools.product(*[p_range, tau_range]): 
            data = ContinousXOR(p=p,tau=tau) 
            partial_estrinsic_results, best_lambda = evaluate_with_lambda_search(model, data, estrinsic_metrics, lambda_range=lambda_range)
            estrinsic_results.append([mask_names[0], N, 0, p, tau ,best_lambda] + partial_estrinsic_results)
        print("Vanilla Evaluated!")

        masks = [
            IPMask.gaussian(N),
            IPMask.bimodal(N, mu=0.85, std=0.15),
            IPMask.trimodal(N, linear_rate=0.4, std_bim=0.18),
            IPMask.trimodal(N, linear_rate=0.2, mu=0.85, std_bim=0.1)
            ]
        
        for m, mask in enumerate(masks):
            config = IPReservoirConfiguration(init_point, mask=mask, epochs=10, name=f"{mask_names[m+1]}")
            configs.append(config)
            model = config.build_up_model(X_TR_INIT)

            partial_intrinsic_results = [mask_names[m+1], N, m+1] + list(np.array([metric.evaluate(model) for metric in intrinsic_metrics]))
            #intrinsic_results.append(partial_intrinsic_results)

            for p,tau in itertools.product(*[p_range, tau_range]): 
                data = ContinousXOR(p=p, tau=tau)
                partial_estrinsic_results, best_lambda = evaluate_with_lambda_search(model, data, estrinsic_metrics, lambda_range=lambda_range)
                estrinsic_results.append([mask_names[m+1], N, m+1, p, tau ,best_lambda] + partial_estrinsic_results)
            print(mask_names[m+1], " Evaluated")

#intrinsic_df = pd.DataFrame(intrinsic_results,columns=intrinsic_column_names) #groupby(by=["Name", "Model Index", "N"])
#intrinsic_df.to_csv("Inubushi-Intrinsic.csv")

estrinsic_df = pd.DataFrame(estrinsic_results,columns=estrinsic_column_names) #groupby(by=["Name", "Model Index", "N", "Log Nu", "Tau", ])
estrinsic_df.to_csv("DambreXOR0-Estrinsic.csv")

Repetiotion: 0
Vanilla Evaluated!
Gaussian  Evaluated
Hard Bimodal  Evaluated
Soft Trimodal  Evaluated
Edgy trimodal  Evaluated
Vanilla Evaluated!
Gaussian  Evaluated
Hard Bimodal  Evaluated
Soft Trimodal  Evaluated
Edgy trimodal  Evaluated
Vanilla Evaluated!
Gaussian  Evaluated
Hard Bimodal  Evaluated
Soft Trimodal  Evaluated
Edgy trimodal  Evaluated
Vanilla Evaluated!
Gaussian  Evaluated
Hard Bimodal  Evaluated
Soft Trimodal  Evaluated
Edgy trimodal  Evaluated
Vanilla Evaluated!
Gaussian  Evaluated
Hard Bimodal  Evaluated
Soft Trimodal  Evaluated
Edgy trimodal  Evaluated
Repetiotion: 1
Vanilla Evaluated!
Gaussian  Evaluated
Hard Bimodal  Evaluated
Soft Trimodal  Evaluated
Edgy trimodal  Evaluated
Vanilla Evaluated!
Gaussian  Evaluated
Hard Bimodal  Evaluated
Soft Trimodal  Evaluated
Edgy trimodal  Evaluated
Vanilla Evaluated!
Gaussian  Evaluated
Hard Bimodal  Evaluated
Soft Trimodal  Evaluated
Edgy trimodal  Evaluated
Vanilla Evaluated!
Gaussian  Evaluated
Hard Bimodal  Evaluated
Sof

In [7]:
estrinsic_df.groupby(by=["Name", "Model Index", "N", "p", "Tau", ]).mean()

Lambda           MSE      NRMSE
Name          Model Index N   p  Tau                                 
Edgy trimodal 4           50  1  3     0.000  1.945245e-04   1.419071
                                 4     0.000  1.227486e-03   3.681740
                                 6     0.000  2.081636e-02  15.201110
                              2  3     0.000  9.159166e-04   7.636104
                                 4     0.000  9.811350e-04   8.565976
...                                      ...           ...        ...
Vanilla       0           250 5  4     0.125  9.915440e-05  22.853819
                                 6     0.200  1.024470e-04  22.819275
                              10 3     0.200  3.140458e-07  22.468195
                                 4     0.200  1.993177e-07  22.340881
                                 6     0.200  3.869799e-07  22.305804

[300 rows x 3 columns]